In [1]:
!pip install soundfile
!pip install numpy
!pip install scipy


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import glob
import soundfile as sf
import numpy as np
from scipy.signal import butter, lfilter
import io  # Import io module for BytesIO

# Function to apply a Butterworth band-pass filter
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

# Function for pre-emphasis
def pre_emphasis(signal, alpha=0.1):
    """Applies pre-emphasis to a signal using a first-order filter."""
    pre_emphasized_signal = lfilter([1, -alpha], [1], signal)
    return pre_emphasized_signal

# Parameters for digitization
lowcut = 300.0  # Lower cutoff frequency for pre-filtering (Hz)
highcut = 3400.0  # Upper cutoff frequency for pre-filtering (Hz)
fs = 8000  # Sampling rate (Hz)0
n_bits = 16  # Quantization accuracy (bits)

# Path to the directory containing folders with .wav files
root_path = "d:\Sistema\Escritorio\Escritorio\Tesis\DAIC-WOZ\data"

# Process each folder and .wav file
for folder_name in os.listdir(root_path):
    folder_path = os.path.join(root_path, folder_name)
    if os.path.isdir(folder_path):
        # Find all .wav files in the current folder
        for wav_file_path in glob.glob(os.path.join(folder_path, '*.wav')):
            # Read the audio signal using soundfile (for efficiency)
            audio_signal, original_fs = sf.read(wav_file_path)

            # Pre-filtering
            filtered_signal = butter_bandpass_filter(audio_signal, lowcut, highcut, original_fs)

            # Sampling (downsampling if necessary)
            if original_fs > fs:
                sampled_signal = filtered_signal[::int(original_fs / fs)]
            else:
                sampled_signal = filtered_signal

            # Pre-emphasis
            pre_emphasized_signal = pre_emphasis(sampled_signal)  # Apply pre-emphasis

            # Quantization (using soundfile for 16-bit quantization)
            with io.BytesIO() as output_bytes:  # Use BytesIO for binary data
                sf.write(output_bytes, pre_emphasized_signal, fs, subtype='PCM_16', format='WAV')
                quantized_signal = output_bytes.getvalue()  # Retrieve quantized data

            # Convert quantized data to NumPy array
            quantized_signal = np.frombuffer(quantized_signal, dtype=np.int16)

            # Coding (example using simple delta encoding)
            if quantized_signal.shape == ():  # If it's a scalar, reshape
                quantized_signal = quantized_signal.reshape(-1)

            coded_signal = np.diff(quantized_signal).astype(np.int16)

            # Extract the audio file name without extension
            audio_file_name = os.path.splitext(os.path.basename(wav_file_path))[0]

            # Output file path
            output_file_path = os.path.join(folder_path, f'preprocessed_{audio_file_name}.wav')

            # Save the digitized audio signal as a .wav file
            sf.write(output_file_path, coded_signal, fs, subtype='PCM_16')

            print(f'Processed and saved: {output_file_path}')  # Print confirmation message


Processed and saved: d:\Sistema\Escritorio\Escritorio\Tesis\DAIC-WOZ\data\300_P\preprocessed_300_AUDIO.wav
Processed and saved: d:\Sistema\Escritorio\Escritorio\Tesis\DAIC-WOZ\data\301_P\preprocessed_301_AUDIO.wav
Processed and saved: d:\Sistema\Escritorio\Escritorio\Tesis\DAIC-WOZ\data\302_P\preprocessed_302_AUDIO.wav
Processed and saved: d:\Sistema\Escritorio\Escritorio\Tesis\DAIC-WOZ\data\303_P\preprocessed_303_AUDIO.wav
Processed and saved: d:\Sistema\Escritorio\Escritorio\Tesis\DAIC-WOZ\data\304_P\preprocessed_304_AUDIO.wav
Processed and saved: d:\Sistema\Escritorio\Escritorio\Tesis\DAIC-WOZ\data\305_P\preprocessed_305_AUDIO.wav
Processed and saved: d:\Sistema\Escritorio\Escritorio\Tesis\DAIC-WOZ\data\306_P\preprocessed_306_AUDIO.wav
Processed and saved: d:\Sistema\Escritorio\Escritorio\Tesis\DAIC-WOZ\data\307_P\preprocessed_307_AUDIO.wav
Processed and saved: d:\Sistema\Escritorio\Escritorio\Tesis\DAIC-WOZ\data\308_P\preprocessed_308_AUDIO.wav
Processed and saved: d:\Sistema\Escri